In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.

In [ ]:
from ortools.sat.python import cp_model

def solve_aircrew_assignment():
    # Crear el modelo
    model = cp_model.CpModel()

    # === DATOS DEL PROBLEMA ===

    # Empleados
    stewards = ['Tom', 'David', 'Jeremy', 'Ron', 'Joe',
                'Bill', 'Fred', 'Bob', 'Mario', 'Ed']
    air_hostesses = ['Carol', 'Janet', 'Tracy', 'Marilyn', 'Carolyn',
                     'Cathy', 'Inez', 'Jean', 'Heather', 'Juliet']

    employees = stewards + air_hostesses
    num_employees = len(employees)
    num_flights = 10

    # Índices de empleados
    steward_indices = list(range(len(stewards)))
    hostess_indices = list(range(len(stewards), len(employees)))

    # Requisitos de tripulación por vuelo
    aircrew_required = [4, 5, 5, 6, 7, 4, 5, 6, 6, 7]
    min_hostesses = [1, 1, 1, 2, 3, 1, 1, 1, 2, 3]
    min_stewards = [1, 1, 1, 2, 3, 1, 1, 1, 2, 3]

    # Conocimiento de idiomas
    french_speakers = ['Inez', 'Bill', 'Jean', 'Juliet']
    spanish_speakers = ['Tom', 'Jeremy', 'Mario', 'Cathy', 'Juliet']
    german_speakers = ['Bill', 'Fred', 'Joe', 'Mario', 'Marilyn',
                       'Inez', 'Heather']

    # Convertir nombres a índices
    french_indices = [employees.index(name) for name in french_speakers]
    spanish_indices = [employees.index(name) for name in spanish_speakers]
    german_indices = [employees.index(name) for name in german_speakers]

    # === VARIABLES DE DECISIÓN ===

    # x[e][f] = 1 si el empleado e está asignado al vuelo f
    x = {}
    for e in range(num_employees):
        for f in range(num_flights):
            x[e, f] = model.NewBoolVar(f'x_e{e}_f{f}')

    # === RESTRICCIONES ===

    # 1. Número exacto de tripulación por vuelo
    for f in range(num_flights):
        model.Add(sum(x[e, f] for e in range(num_employees)) == aircrew_required[f])

    # 2. Número mínimo de azafatas por vuelo
    for f in range(num_flights):
        model.Add(sum(x[e, f] for e in hostess_indices) >= min_hostesses[f])

    # 3. Número mínimo de sobrecargos por vuelo
    for f in range(num_flights):
        model.Add(sum(x[e, f] for e in steward_indices) >= min_stewards[f])

    # 4. Restricciones de idiomas: al menos uno por idioma en cada vuelo
    for f in range(num_flights):
        # Francés
        model.Add(sum(x[e, f] for e in french_indices) >= 1)
        # Español
        model.Add(sum(x[e, f] for e in spanish_indices) >= 1)
        # Alemán
        model.Add(sum(x[e, f] for e in german_indices) >= 1)

    # 5. No asignar a vuelos consecutivos
    for e in range(num_employees):
        for f in range(num_flights - 1):
            model.Add(x[e, f] + x[e, f + 1] <= 1)

    # === RESOLVER ===

    solver = cp_model.CpSolver()
    solver.parameters.log_search_progress = True
    solver.parameters.max_time_in_seconds = 60.0

    status = solver.Solve(model)

    # === MOSTRAR RESULTADOS ===

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("\n" + "="*80)
        print("SOLUCIÓN ENCONTRADA")
        print("="*80)

        if status == cp_model.OPTIMAL:
            print("Estado: ÓPTIMA")
        else:
            print("Estado: FACTIBLE")

        print(f"\nTiempo de resolución: {solver.WallTime():.2f} segundos")
        print(f"Número de branches explorados: {solver.NumBranches()}")

        # Mostrar asignaciones por vuelo
        print("\n" + "-"*80)
        print("ASIGNACIONES POR VUELO")
        print("-"*80)

        for f in range(num_flights):
            print(f"\n📍 VUELO {f + 1}:")
            print(f"   Tripulación requerida: {aircrew_required[f]}")

            assigned_employees = []
            assigned_stewards = []
            assigned_hostesses = []
            languages = {'Francés': [], 'Español': [], 'Alemán': []}

            for e in range(num_employees):
                if solver.Value(x[e, f]) == 1:
                    emp_name = employees[e]
                    assigned_employees.append(emp_name)

                    if e in steward_indices:
                        assigned_stewards.append(emp_name)
                    else:
                        assigned_hostesses.append(emp_name)

                    # Verificar idiomas
                    if emp_name in french_speakers:
                        languages['Francés'].append(emp_name)
                    if emp_name in spanish_speakers:
                        languages['Español'].append(emp_name)
                    if emp_name in german_speakers:
                        languages['Alemán'].append(emp_name)

            print(f"   Sobrecargos ({len(assigned_stewards)}): {', '.join(assigned_stewards)}")
            print(f"   Azafatas ({len(assigned_hostesses)}): {', '.join(assigned_hostesses)}")
            print(f"   Idiomas:")
            for lang, speakers in languages.items():
                print(f"      - {lang}: {', '.join(speakers)}")

        # Verificar restricciones
        print("\n" + "-"*80)
        print("VERIFICACIÓN DE RESTRICCIONES")
        print("-"*80)

        all_constraints_met = True

        for f in range(num_flights):
            assigned = [e for e in range(num_employees) if solver.Value(x[e, f]) == 1]
            num_stewards_assigned = sum(1 for e in assigned if e in steward_indices)
            num_hostesses_assigned = sum(1 for e in assigned if e in hostess_indices)

            has_french = any(employees[e] in french_speakers for e in assigned)
            has_spanish = any(employees[e] in spanish_speakers for e in assigned)
            has_german = any(employees[e] in german_speakers for e in assigned)

            if len(assigned) != aircrew_required[f]:
                print(f"❌ Vuelo {f+1}: Tripulación incorrecta")
                all_constraints_met = False
            elif num_stewards_assigned < min_stewards[f]:
                print(f"❌ Vuelo {f+1}: Insuficientes sobrecargos")
                all_constraints_met = False
            elif num_hostesses_assigned < min_hostesses[f]:
                print(f"❌ Vuelo {f+1}: Insuficientes azafatas")
                all_constraints_met = False
            elif not (has_french and has_spanish and has_german):
                print(f"❌ Vuelo {f+1}: Faltan idiomas requeridos")
                all_constraints_met = False

        # Verificar vuelos consecutivos
        for e in range(num_employees):
            for f in range(num_flights - 1):
                if solver.Value(x[e, f]) == 1 and solver.Value(x[e, f + 1]) == 1:
                    print(f"❌ {employees[e]} asignado a vuelos consecutivos {f+1} y {f+2}")
                    all_constraints_met = False

        if all_constraints_met:
            print("✅ Todas las restricciones se cumplen correctamente")

        # Mostrar carga de trabajo por empleado
        print("\n" + "-"*80)
        print("CARGA DE TRABAJO POR EMPLEADO")
        print("-"*80)

        for e in range(num_employees):
            flights_assigned = [f+1 for f in range(num_flights) if solver.Value(x[e, f]) == 1]
            if flights_assigned:
                print(f"{employees[e]:12} → Vuelos: {flights_assigned}")

    elif status == cp_model.INFEASIBLE:
        print("\n❌ El problema no tiene solución (INFEASIBLE)")
    elif status == cp_model.MODEL_INVALID:
        print("\n❌ El modelo es inválido")
    else:
        print(f"\n⚠️  Estado desconocido: {status}")

    return status, solver if status in [cp_model.OPTIMAL, cp_model.FEASIBLE] else None

# Ejecutar el solver
if __name__ == "__main__":
    print("="*80)
    print("PROBLEMA DE ASIGNACIÓN DE TRIPULACIÓN AÉREA")
    print("Modelo CSP usando OR-Tools CP-SAT Solver")
    print("="*80)

    status, solver = solve_aircrew_assignment()

PROBLEMA DE ASIGNACIÓN DE TRIPULACIÓN AÉREA
Modelo CSP usando OR-Tools CP-SAT Solver

SOLUCIÓN ENCONTRADA
Estado: ÓPTIMA

Tiempo de resolución: 0.03 segundos
Número de branches explorados: 1416

--------------------------------------------------------------------------------
ASIGNACIONES POR VUELO
--------------------------------------------------------------------------------

📍 VUELO 1:
   Tripulación requerida: 4
   Sobrecargos (3): Joe, Bill, Mario
   Azafatas (1): Jean
   Idiomas:
      - Francés: Bill, Jean
      - Español: Mario
      - Alemán: Joe, Bill, Mario

📍 VUELO 2:
   Tripulación requerida: 5
   Sobrecargos (2): Jeremy, Ron
   Azafatas (3): Janet, Tracy, Inez
   Idiomas:
      - Francés: Inez
      - Español: Jeremy
      - Alemán: Inez

📍 VUELO 3:
   Tripulación requerida: 5
   Sobrecargos (1): Fred
   Azafatas (4): Carol, Marilyn, Carolyn, Juliet
   Idiomas:
      - Francés: Juliet
      - Español: Juliet
      - Alemán: Fred, Marilyn

📍 VUELO 4:
   Tripulación requeri